In [1]:
import sys
sys.path.append("..")

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.model_selection import StratifiedKFold,cross_val_score

from data_preprocessing.preprocessor import Preprocessor

sns.set_style('darkgrid')
pd.set_option('display.max_columns',500)

In [3]:
df = pd.read_csv('../datasets/final_combined_dataset.csv')

In [4]:
preprocessor = Preprocessor(df)
preprocessor.remove_features()

In [37]:
X_nan = df[df.isnull().T.any()].drop('FLAG',axis=1)
y_nan = df[df.isnull().T.any()]['FLAG']

In [38]:
df_dropped = df.dropna()

In [39]:
X,y = df_dropped.drop('FLAG',axis=1),df_dropped['FLAG']

In [40]:
skf = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)

In [41]:
for train_index,test_index in skf.split(X,y):
    X_train,X_test = X.iloc[train_index],X.iloc[test_index],
    y_train,y_test = y.iloc[train_index],y.iloc[test_index]

In [56]:
y_train_combined = np.concatenate([y_train,y_nan])
X_train_combined = pd.concat([X_train,X_nan])

In [128]:
xgb = XGBClassifier(n_estimators=200,max_depth=4,random_state=42)
xgb.fit(X_train_combined,y_train_combined)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=250, n_jobs=0, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [129]:
print(cross_val_score(xgb,X_train_combined,y_train_combined,cv=skf,scoring='f1').mean())
print(f1_score(y_test,predictions))

0.9750178810190118
0.9655172413793104


In [121]:
predictions = xgb.predict(X_test)

In [122]:
from sklearn.metrics import f1_score,classification_report,confusion_matrix

0.9655172413793104
